## Basis order

$$
|qubit_1\rangle \otimes |qubit_2\rangle \otimes |vib_1\rangle \otimes |vib_2\rangle \otimes |cav\rangle
$$

In [1]:
import numpy as np
import scipy.linalg as la
import qutip
from qutip import *

#define fock states
N_q1 = 2
N_q2 = 2
N_vib1 = 2
N_vib2 = 2
N_cav = 2

wc = 1.0       # cav frequency
w1 = 1.0       # q1 electronic frequency
w2 = 1.0       # q2 electronic frequency
wv1 = 0.1      # q1 vibrational frequency
wv2 = 0.1      # q2 vibrational frequency
g1 = 0.1       # coupling strength (in arbitrary units)
g2 = 0.1       # coupling strength (in arbitrary units)
lam1 = 0.1     # sqrt(Huang-Rhys) of q1
lam2 = 0.1     # sqrt(Huang-Rhys) of q2

# Define the Pauli matrices for the atom Hilbert space (N = 2)
sigmaz = sigmaz()
sigmap = sigmap()
sigmam = sigmam()

Iq1 = qeye(N_q1)
Iq2 = qeye(N_q2)
Ivib1 = qeye(N_vib1)
Ivib2 = qeye(N_vib2)
Icav = qeye(N_cav)

#cavity operators
am = tensor(Iq1, Iq2, Ivib1, Ivib2, destroy(N_cav))
ap = am.dag()

#vib1 operators
bm1 = tensor(Iq1, Iq2, destroy(N_vib1), Ivib2, Icav)
bp1 = bm1.dag()

#vib2 operators
bm2 = tensor(Iq1, Iq2, Ivib1, destroy(N_vib2), Icav)
bp2 = bm2.dag()

#q1 operators
sm1 = tensor(sigmap, Iq2, Ivib1, Ivib2, Icav)
sp1 = sm1.dag()

#q2 operators
sm2 = tensor(Iq1, sigmap, Ivib1, Ivib2, Icav)
sp2 = sm2.dag()

#sigma_z operators
sz1 = tensor(sigmaz, Iq2, Ivib1, Ivib2, Icav)
sz2 = tensor(Iq1, sigmaz, Ivib1, Ivib2, Icav)

In [2]:
# #build hamiltonians
# H_atom = - w / 2 * sz #
# H_vib = wv * bp * bm #
# H_qubit_vib = - lam * wv * (sp * bm  + sm * bp) #
# H_qubit_vib = - lam * wv * (bp + bm) * sp * sm
# H_cav = wc * ap * am #
# H_qubit_cav = g * (am *sp + ap * sm) #

#building Hamiltonians
H_q1 = - w1 / 2 * sz1
H_q2 = - w2 / 2 * sz2
H_vib1 = wv1 * bp1 * bm1
H_vib2 = wv2 * bp2 * bm2
# H_qubit_vib1 = - lam1 * wv1 * (sp1 * bm1 + sm1 * bp1)
# H_qubit_vib2 = - lam2 * wv2 * (sp2 * bm2 + sm2 * bp2)
H_qubit_vib1 = - lam1 * wv1 * (bp1 + bm1) * sp1 * sm1
H_qubit_vib2 = - lam2 * wv2 * (bp2 + bm2) * sp2 * sm2
H_cav = wc * ap * am
H_qubit_cav1 = g1 * (am * sp1 + ap * sm1)
H_qubit_cav2 = g2 * (am * sp2 + ap * sm2)

In [3]:
#build full Hamiltonians
H_change = H_q1 + H_q2 + H_vib1 + H_vib2 + H_qubit_vib1 + H_qubit_vib2 + H_cav
H_htc = H_change + H_qubit_cav1 + H_qubit_cav2

# Compute eigenvalues and eigenvectors of the Hamiltonian
eigenvalues, eigenvecs = H_change.eigenstates()

eigenvectors = Qobj(np.column_stack([vec.full().real for vec in eigenvecs]), dims=H_change.dims)

# Transform JC Hamiltonian to get H_polaron (polariton basis representation)
H_polaron = eigenvectors.dag() * H_htc * eigenvectors

#build diagonal matrix
H_diag = Qobj(np.diag(eigenvalues),dims=H_change.dims)

In [4]:
eigenvectors

Quantum object: dims=[[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape=(32, 32), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -9.81276209e-01
   1.33664627e-01 -9.80580676e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -4.18956905e-17
  -3.36206652e-17 -3.63276896e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.09567443e-01
  -8.50043805e-02  9.90290338e-01]]

In [5]:
H_diag

Quantum object: dims=[[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape=(32, 32), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -0.9         0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.9        ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  2.1         0.
   0.        ]
 [ 0.          0.          0.         ...  0.          2.1
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   2.20198039]]

In [6]:
np.allclose((eigenvectors.dag() * H_change * eigenvectors).full(), H_diag.full())

True

In [7]:
eigenvectors.full().real[:,29]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.58287054e-19,  1.09567443e-01, -3.83387695e-16, -1.14398220e-01,
        2.82746120e-16, -9.81276209e-01, -4.18956905e-17, -1.09567443e-01])

In [8]:
basis_states = []

j = 0
for i in eigenvecs:
    n = 0
    while n < (N_q1 * N_q2 * N_vib1 * N_vib2 * N_cav):
        if np.allclose(i.full().real, basis((N_q1 * N_q2 * N_vib1 * N_vib2 * N_cav), n).full().real):
            basis_states.append([i, f"{j}th", f"{n} basis"])
            
        n += 1
    j += 1

In [9]:
len(basis_states)

8

In [10]:
np.allclose(( bp2 * basis_states[0][0]).full().real, basis_states[1][0].full().real)

True

In [11]:
basis_states

[[Quantum object: dims=[[2, 2, 2, 2, 2], [1]], shape=(32, 1), type='ket', dtype=Dense
  Qobj data =
  [[1.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]],
  '0th',
  '0 basis'],
 [Quantum object: dims=[[2, 2, 2, 2, 2], [1]], shape=(32, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.]
   [0.]
   [1.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]],
  '1th',
  '2 basis'],
 [Quantum object: dims=[[2, 2, 2, 2, 2], [1]], shape=(32, 1), type='ket', dtype=Dense
  Qobj data =
  [[0.]
   [0.]
   [0.]
   [0.]
   [1.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
  

In [12]:
def build_coff_expansion(vec):
    n = 0
    coff = 0
    while n < len(vec):
        coff += eigenvecs[n] * vec[n]
        n += 1

    return coff

In [13]:
n = 0
fock = N_q1 * N_q2 * N_vib1 * N_vib2 * N_cav

while n < fock:
    print(np.allclose(basis(fock, n).full().real, build_coff_expansion(eigenvectors.full().real[n,:]).full().real, rtol=1e-8))
    n += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [14]:
H_exp_uncoupled = expect(H_htc, eigenvecs[31])
H_exp_uncoupled

2.201980390271856

In [15]:
H_diag.diag()[31]

np.float64(2.2019803902718555)

In [16]:
n = 0
while n < fock:
    print(n, np.isclose(expect(H_htc, eigenvecs[n]), H_diag.diag()[n], atol=1e-5))
    n += 1

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True


In [17]:
def find_ket_values(tensor_order, basis_state):
    """Find the ket values for a given basis state and tensor order.
    Args:
        tensor_order (list): The order of the tensor product and number of states for each qubit
        basis_state (ndarray): The basis state as an ndarray."""
    
    ket_values = []
    entries = []

    #get the entries of the basis state that are not zero
    i = 0
    for n in basis_state:
        if n != 0:
            entries.append([n[0],i])
        i += 1
    
    basis_size = 1
    for i in tensor_order:
        basis_size *= i

    index = -1
    for i in entries:
        basis = basis_size
        ket_values.append([])
        start_val = 0
        index += 1
        for n in tensor_order:
            order = 0
            tem = basis /n 
            temp = tem + start_val
            while True:
                if i[1] < temp:
                    ket_values[index].append(order)
                    basis = tem
                    break
                else:
                    start_val += tem
                    temp += tem
                    order += 1
    
    return ket_values, entries

In [18]:
ket_order = [N_q1, N_q2, N_vib1, N_vib2, N_cav]
find_ket_values(ket_order, eigenvecs[31].full().real)

([[1, 1, 0, 0, 0],
  [1, 1, 0, 0, 1],
  [1, 1, 0, 1, 0],
  [1, 1, 0, 1, 1],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 1],
  [1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1]],
 [[np.float64(4.8483833335279055e-20), 24],
  [np.float64(0.009709662154539986), 25],
  [np.float64(-3.927644878854764e-17), 26],
  [np.float64(-0.0980580675690923), 27],
  [np.float64(3.1847213726501453e-17), 28],
  [np.float64(-0.09805806756909238), 29],
  [np.float64(-3.632768961620567e-18), 30],
  [np.float64(0.9902903378454601), 31]])

In [19]:
ket_order = [N_q1, N_q2, N_vib1, N_vib2, N_cav]
find_ket_values(ket_order, basis_states[0][0].full().real)

([[0, 0, 0, 0, 0]], [[np.float64(1.0), 0]])

In [20]:
find_ket_values(ket_order, basis(fock, 14).full().real)

([[0, 1, 1, 1, 0]], [[np.float64(1.0), 14]])

```python
kets = find_ket_values(tensor_order, basis)
```